<a href="https://colab.research.google.com/github/AI-Isaiah/AI-Isaiah/blob/main/20230329_9_ensemble_method_with_smote_optuna%2Bcost_function_AFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#!pip install optuna
#!pip install catboost
import seaborn as sns
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
import sys
sys.path.append('/bin/sample_data')
sys.path.append('/root')
#from preprocessor import generate_cm
from collections import Counter
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
import optuna

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV, ParameterSampler, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_auc_score, roc_curve, recall_score, accuracy_score, make_scorer, classification_report
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import gc
import json
from imblearn.over_sampling import SMOTE, ADASYN, SMOTENC


In [41]:
!nvidia-smi

Wed Apr  5 06:42:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    29W /  70W |    205MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [62]:
def generate_cm(y_true, y_pred, title = ""):
    y_pred = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    labels = ['True', 'False']
    sns.set(font_scale=0.8) # adjust font size
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)

    # Add labels and title to the plot
    plt.ylabel('Predicted Labels')
    plt.xlabel('True Labels - updated')
    plt.title('Confusion Matrix: ' + title)
    plt.show()

    print("Counter" + title, Counter(y_pred))

In [42]:
data = pd.read_csv("/content/sample_data/loan_book_data_labels.csv")
labels = data["Result"]
data.drop("Result", axis=1, inplace=True)
print(labels.head())

0    1
1    1
2    1
3    1
4    1
Name: Result, dtype: int64


In [72]:
def objective_xgboost1(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'gamma': trial.suggest_float('gamma', 1e-9, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "random_state": 42,
    }
            
    
    # Create an XGBClassifier with the specified hyperparameters
    clf = xgb.XGBClassifier(**params, tree_method="gpu_hist", enable_categorical=True, max_cat_to_onehot=1)
    clf.fit(X, y, verbose=False)
    y_pred = clf.predict(X)
    y_pred = (y_pred > 0.5).astype(int)
    return accuracy_score(y, y_pred)

def objective_low_level(trial, X, y):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'gamma': trial.suggest_float('gamma', 1e-9, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "random_state": 42,
    }
 
    # Create an XGBClassifier with the specified hyperparameters
    #clf = xgb.XGBClassifier(**params,enable_categorical=True)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    #X = X.values   
    
    dtrain = xgb.DMatrix(X, y, enable_categorical=True)
    
    bst = xgb.train(params, dtrain, 500, verbose_eval=False, tree_method="gpu_hist", max_cat_to_onehot=1)
    #scores = cross_val_score(bst, X, y, cv=skf, scoring='accuracy')

    y_pred = bst.predict(dtrain) #ntree_limit=bst.best_ntree_limit)
        
    y_pred = (y_pred > 0.5).astype(int)
    scores.append(accuracy_score(y, y_pred))
    #params['n_estimators'] = num_round


    
    return np.mean(scores)

   

def feature_selection2(X, y, max_features=30, n_trials=50):
    # Split data for hyperparameter tuning
    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
     # Create a DMatrix for training
    """dtrain = xgb.DMatrix(data=X_train, label=y_train, enable_categorical=True)
    dtest = xgb.DMatrix(data=X_valid, label=y_valid, enable_categorical=True)
    """
    # Optimize hyperparameters using Optuna
    
    study = optuna.create_study(direction='maximize', sampler = optuna.samplers.NSGAIISampler(seed = 42)) # update2
    study.optimize(lambda trial: objective_xgboost1(trial, X,y), n_trials=n_trials)
    best_params = study.best_params
    print('Best hyperparameters: {}'.format(best_params))

    # Train initial XGBoost model with the best hyperparameters
    #num_round = best_params.pop('n_estimators')  # Remove n_estimators from params and store its value
    #init_clf = xgb.train(best_params, dtrain, num_round, [(dtest, 'validation')], early_stopping_rounds=100, verbose_eval=False)
    init_clf = xgb.XGBClassifier(**best_params, tree_method="gpu_hist", enable_categorical=True, max_cat_to_onehot=1)
    init_clf.fit(X,y)

    # Get feature importances and indices
    """importances = init_clf.get_score(importance_type='weight')
    indices = sorted(importances, key=lambda x: importances[x], reverse=True)
    """
    importances = init_clf.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Select the top-k features
    selected_features = indices[:max_features]
    print("Indices in function: ", indices)
    #X_selected = X[:, selected_features]
    

    return selected_features



In [44]:
#X = data
X = data.drop("v01_src", axis=1)
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
print(X.dtypes)
feature_names = X.columns
le = LabelEncoder()
y = le.fit_transform(labels)
y = y.ravel()
cat_columns = X.select_dtypes(include='category').columns

num_columns = X.select_dtypes(exclude=['category']).columns

# Encode categorical columns using LabelEncoder
"""label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le"""



v02_src    category
v03_src       int64
v04_src    category
v05_src    category
v06_src    category
             ...   
v65_src       int64
v66_src     float64
v67_src     float64
v68_src       int64
v69_src       int64
Length: 68, dtype: object


'label_encoders = {}\nfor col in cat_columns:\n    le = LabelEncoder()\n    X[col] = le.fit_transform(X[col])\n    label_encoders[col] = le'

In [45]:
xgb.__version__

'1.7.5'

In [73]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

X_train = pd.DataFrame(X_train, columns=X.columns)
y_train = pd.DataFrame(y_train, columns=["Result"])
X_val = pd.DataFrame(X_val, columns=X.columns)
y_val = pd.DataFrame(y_val, columns=["Result"])
X_test = pd.DataFrame(X_test, columns=X.columns)
y_test = pd.DataFrame(y_test, columns=["Result"])


selected_features = feature_selection2(X_train, y_train, 30, 50)



cat_columns
X_train = X_train.iloc[:, selected_features]
X_val = X_val.iloc[:, selected_features]
X_test = X_test.iloc[:, selected_features]

# Identify categorical columns
cat_cols = X_train.select_dtypes(include=['category']).columns.tolist()
print("Cat-cols: ", cat_cols)
num_cols = X_train.select_dtypes(exclude=['category']).columns.tolist()
# Create a SMOTENC object with categorical features
smote_nc = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in cat_cols], random_state=42)

# Fit the SMOTENC object to the training data
X_train, y_train = smote_nc.fit_resample(X_train, y_train)

print("X_train_shape final: ", X_train.shape)


[I 2023-04-05 07:57:16,717] A new study created in memory with name: no-name-fcfa91fe-625f-4a6d-8956-a0d631daa791
[I 2023-04-05 07:57:17,408] Trial 0 finished with value: 0.8600973236009732 and parameters: {'n_estimators': 381, 'max_depth': 10, 'learning_rate': 0.7322619478695936, 'gamma': 59.865848420104996, 'subsample': 0.24041677639819287, 'colsample_bytree': 0.2403950683025824, 'reg_alpha': 5.808361217761862, 'reg_lambda': 86.61761457762734, 'min_child_weight': 7}. Best is trial 0 with value: 0.8600973236009732.
[I 2023-04-05 07:57:18,691] Trial 1 finished with value: 0.8600973236009732 and parameters: {'n_estimators': 711, 'max_depth': 3, 'learning_rate': 0.9699399423098324, 'gamma': 83.24426408020973, 'subsample': 0.29110519961044856, 'colsample_bytree': 0.26364247048639056, 'reg_alpha': 18.340450986159976, 'reg_lambda': 30.42422429664953, 'min_child_weight': 6}. Best is trial 0 with value: 0.8600973236009732.


Best hyperparameters: {'n_estimators': 381, 'max_depth': 10, 'learning_rate': 0.7322619478695936, 'gamma': 59.865848420104996, 'subsample': 0.24041677639819287, 'colsample_bytree': 0.2403950683025824, 'reg_alpha': 5.808361217761862, 'reg_lambda': 86.61761457762734, 'min_child_weight': 7}
Indices in function:  [67 24 18 19 20 21 22 23 25 66 26 27 28 29 30 31 17 16 15 14 13 12 11 10
  9  8  7  6  5  4  3  2  1 32 33 34 51 65 64 63 62 61 60 59 58 57 56 55
 54 53 52 50 35 49 48 47 46 45 44 43 42 41 40 39 38 37 36  0]
Cat-cols:  ['v20_src', 'v21_src', 'v22_src', 'v19_src', 'v18_src', 'v13_src', 'v12_src', 'v10_src', 'v07_src', 'v06_src']
X_train_shape final:  (1414, 30)


In [57]:


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols)
    ],
    remainder='passthrough'
)

# Fit the preprocessor on the training data
X_train= preprocessor.fit_transform(X_train)

# Apply the preprocessor on the validation and test data
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)




In [ ]:

def custom_loss(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (tn * 4) - (fp * 10)

# Define the objective function for Optuna
def objective_catboost(trial, X, y, cat_cols):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 5, 200),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.00),
        'sampling_frequency': trial.suggest_categorical('sampling_frequency', ["PerTree", "PerTreeLevel"]),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 10),
        'task_type': 'GPU'
    }
    
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Train and evaluate the model using cross-validation
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        # Create a Pool object with categorical features
        pool_train = Pool(X_train, y_train, cat_features=cat_cols)
        pool_valid = Pool(X_val, y_val, cat_features=cat_cols)
        
        # Train the model
        model = CatBoostClassifier(**params)
        model.fit(pool_train, eval_set=pool_valid, verbose=False)
        
        # Evaluate the model on the validation set
        y_pred = model.predict(X_val)
        y_pred = (y_pred > 0.5).astype(int)
        score = custom_loss(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)


# Optimize hyperparameters
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.NSGAIISampler(seed = 42))
study.optimize(lambda trial: objective_catboost(trial, X_train,y_train, cat_cols), n_trials=100)

# Get best parameters
best_params = study.best_params
print("Best parameters: ", best_params)

#Pool variables to be used with catboost:
pool_train = Pool(X_train, y_train, cat_features = cat_cols)
pool_valid = Pool(X_val, cat_features = cat_cols)
pool_test = Pool(X_test, cat_features = cat_cols)


# Run the randomized search
cb_classifier = CatBoostClassifier(**best_params, random_state=42, verbose=0)
cb_classifier.fit(pool_train)
tn, fp, fn, tp = confusion_matrix(y_val, cb_classifier.predict(X_val)).ravel()
print("confusion Matrix validation")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_val, cb_classifier.predict(pool_valid), "CatBoost - Validation")
tn, fp, fn, tp = confusion_matrix(y_test, cb_classifier.predict(X_test)).ravel()
print("confusion Matrix test")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_test, cb_classifier.predict(pool_test), "CatBoost - Test")

# Save the best parameters for CatBoost
with open('cat_best_params.json', 'w') as f:
    json.dump(study.best_params, f)

# Clear memory
del cb_classifier
gc.collect()



[I 2023-04-05 08:46:43,532] A new study created in memory with name: no-name-8e4335b4-01c5-47d6-b84c-4eea4c692b16


In [ ]:
# Create a RandomForest classifier
rf_clf = RandomForestClassifier(random_state=42)

# Define the parameter space for RandomForest
rf_param_grid = {
    'n_estimators': list(range(50, 500)),
    'max_depth': list(range(1, 50)),
    'min_samples_split': list(range(2, 50)),
    'min_samples_leaf': list(range(1, 50)),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
}

# Set up randomized search for RandomForest
rf_random_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=rf_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

rf_random_search.fit(X_train, y_train)

generate_cm(y_val, rf_random_search.predict(X_val))
generate_cm(y_test, rf_random_search.predict(X_test))

# Save the best parameters for Logistic Regression
with open('rf_best_params.json', 'w') as f:
    json.dump(rf_random_search.best_params_, f)

# Clear memory
del rf_random_search
gc.collect()


In [ ]:
# Create a Logistic Regression classifier
logreg_clf = LogisticRegression(random_state=42, max_iter=1000)

# Define the parameter space for Logistic Regression
logreg_param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'None'],
    'C': list(np.logspace(-4, 4, 1000)),
    'fit_intercept': [True, False],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'l1_ratio': list(np.linspace(0, 1, 1000)),
}

# Set up randomized search for Logistic Regression
logreg_random_search = RandomizedSearchCV(
    estimator=logreg_clf,
    param_distributions=logreg_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

# Run the randomized search
logreg_random_search.fit(X_train, y_train)

generate_cm(y_val, logreg_random_search.predict(X_val))
generate_cm(y_test, logreg_random_search.predict(X_test))

# Save the best parameters for Logistic Regression
with open('logreg_best_params.json', 'w') as f:
    json.dump(logreg_random_search.best_params_, f)

# Clear memory
del logreg_random_search
gc.collect()


In [ ]:
# Create an SVM classifier
svm_clf = SVC(random_state=42, probability=True)

# Define the parameter space for SVM
svm_param_grid = {
    'C': list(np.logspace(-4, 4, 1000)),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(range(1, 6)),
    'gamma': ['scale', 'auto'] + list(np.logspace(-4, 1, 1000)),
    'coef0': list(np.linspace(0, 1, 1000)),
    'shrinking': [True, False],
}

# Set up randomized search for SVM
svm_random_search = RandomizedSearchCV(
    estimator=svm_clf,
    param_distributions=svm_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

# Run the randomized search
svm_random_search.fit(X_train, y_train)

# Save the best parameters for SVM
with open('svm_best_params.json', 'w') as f:
    json.dump(svm_random_search.best_params_, f)

generate_cm(y_val, svm_random_search.predict(X_val))
generate_cm(y_test, svm_random_search.predict(X_test))

# Clear memory
del svm_random_search
gc.collect()


In [ ]:
print("X_train shape: ", X_train.shape)
print("X_val shape: ", X_val.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
lgbm_clf = LGBMClassifier(random_state=42)
lgbm_params = {
    'learning_rate': np.logspace(-3, 0, 10),
    'n_estimators': range(50, 500),
    'num_leaves': range(20, 100),
    'min_child_samples': range(5, 50),
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
    'subsample': np.linspace(0.1, 1, 10),
    'colsample_bytree': np.linspace(0.1, 1, 10),
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]
}

recall_scorer = make_scorer(recall_score, greater_is_better=True)

lgbm_random_search = RandomizedSearchCV(lgbm_clf, param_distributions=lgbm_params, n_iter=50, scoring=recall_scorer, cv=5, n_jobs=-1, verbose=2, random_state=42)
lgbm_random_search.fit(X_train, y_train)

lgbm_best_params = lgbm_random_search.best_params_
print(lgbm_best_params)

generate_cm(y_val, lgbm_random_search.predict(X_val))
generate_cm(y_test, lgbm_random_search.predict(X_test))

with open('lgbm_best_params.json', 'w') as f:
    json.dump(lgbm_best_params, f)



In [ ]:
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1)
xgb_params = {
    'learning_rate': np.logspace(-3, 0, 10),
    'n_estimators': range(50, 500),
    'max_depth': range(3, 15),
    'min_child_weight': range(1, 10),
    'gamma': np.linspace(0, 1, 11),
    'subsample': np.linspace(0.1, 1, 10),
    'colsample_bytree': np.linspace(0.1, 1, 10),
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]
}

xgb_random_search = RandomizedSearchCV(xgb_clf, param_distributions=xgb_params, n_iter=50, scoring=recall_scorer, cv=5, n_jobs=-1, verbose=2, random_state=42)
xgb_random_search.fit(X_train, y_train)

xgb_best_params = xgb_random_search.best_params_
print(xgb_best_params)

with open('xgb_best_params.json', 'w') as f:
    json.dump(xgb_best_params, f)

generate_cm(y_val, xgb_random_search.predict(X_val), "XGBoost, Randomized Search, Validation Set")
generate_cm(y_test, xgb_random_search.predict(X_test), "XGBoost, Randomized Search, Validation Set")

print("Counter - valid" , Counter(y_val))
print("Counter - test", Counter(y_test))



In [ ]:
from sklearn.ensemble import VotingClassifier

# Load the best parameters for each classifier
with open('lgbm_best_params.json', 'r') as f:
    lgbm_best_params = json.load(f)

with open('xgb_best_params.json', 'r') as f:
    xgb_best_params = json.load(f)

with open('cat_best_params.json', 'r') as f:
    cat_best_params = json.load(f)

with open('logreg_best_params.json', 'r') as f:
    logreg_best_params = json.load(f)

with open('rf_best_params.json', 'r') as f:
    rf_best_params = json.load(f)

with open('svm_best_params.json', 'r') as f:
    svm_best_params = json.load(f)




In [ ]:
# Instantiate classifiers with the best parameters
lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, verbose = False, **cat_best_params)
logreg_clf = LogisticRegression(random_state=42, **logreg_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
svm_clf = SVC(probability=True, random_state=42, **svm_best_params)

# Create a soft-voting classifier
ensemble_clf = VotingClassifier(estimators=[
    ('lgbm', lgbm_clf),
    ('xgb', xgb_clf),
    ('cat', cat_clf),
    ('logreg', logreg_clf),
    ('rf', rf_clf),
    ('svm', svm_clf)
], voting='soft')



# Train the ensemble classifier
ensemble_clf.fit(X_train, y_train)

# Calculate the recall score for the ensemble
ensemble_recall = recall_score(y_true=y_test, y_pred=ensemble_clf.predict(X_test), pos_label=1)
print(f"Ensemble recall score: {ensemble_recall}")

generate_cm(y_val, ensemble_clf.predict(X_val))
generate_cm(y_test, ensemble_clf.predict(X_test))


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)

In [ ]:
precision = precision_score(y_true=y_test, y_pred=y_pred, pos_label=1)
recall = recall_score(y_true=y_test, y_pred=y_pred, pos_label=1)
f1 = f1_score(y_true=y_test, y_pred=y_pred, pos_label=1)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


In [ ]:
y_pred_proba = ensemble_clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)

fpr, tpr, _ = roc_curve(y_true=y_test, y_score=y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"Ensemble Classifier (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


Addition of feauture reduction methods

In [ ]:

import os
from contextlib import redirect_stdout

ensemble_clf.fit(X_train, y_train)

def voting_classifier_feature_importances(voting_clf):
    importances = []
    for clf in voting_clf.estimators_:
        if isinstance(clf, tuple):  # Check if the clf is a tuple (name, estimator)
            clf = clf[1]
        if hasattr(clf, 'coef_'):
            importances.append(clf.coef_.flatten())
        elif hasattr(clf, 'feature_importances_'):
            importances.append(clf.feature_importances_)
    
    if not importances:
        raise ValueError("No estimators in the ensemble have `coef_` or `feature_importances_` attribute.")
    
    return np.mean(importances, axis=0)


rfe = RFECV(estimator=ensemble_clf, step=1, cv=5, scoring='recall', n_jobs=-1, importance_getter=voting_classifier_feature_importances)

with open(os.devnull, 'w') as devnull:
    with redirect_stdout(devnull):
        rfe.fit(X_train, y_train)

# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)
# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
ensemble_clf.fit(X_train_rfe, y_train)
y_pred_rfe = ensemble_clf.predict(X_test_rfe)

accuracy_rfe = accuracy_score(y_test, y_pred_rfe)
report_rfe = classification_report(y_test, y_pred_rfe)
conf_matrix_rfe = confusion_matrix(y_test, y_pred_rfe)

print(f"Accuracy (with RFE): {accuracy_rfe:.4f}")
print("Classification Report (with RFE):")
print(report_rfe)
print("Confusion Matrix (with RFE):")
print(conf_matrix_rfe)


Use stacking method to combine the results of different models
Extra trees classifier and Principal component analysis

In [ ]:
ex_clf = ExtraTreesClassifier(random_state=42)

pipeline = Pipeline([
    ('pca', PCA()),
    ('ex_clf', ex_clf),
])
param_grid = {
    'pca__n_components': np.arange(1, X_train.shape[1] + 1),
    'ex_clf__n_estimators': [100, 200, 500],
    'ex_clf__max_depth': [None, 10, 20, 30],
    'ex_clf__min_samples_split': [2, 5, 10],
    'ex_clf__min_samples_leaf': [1, 2, 4],
}

random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=50, cv=5, scoring='recall', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)
ex_clf_best_params = random_search.best_params_

y_pred = random_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(range(X_train.shape[1]), importances[sorted_indices])
plt.xticks(range(X_train.shape[1]), sorted_indices)
plt.xlabel("Feature index")
plt.ylabel("Importance")
plt.title("Feature importances in the best ExtraTreesClassifier model")
plt.show()


Use the other classifiers to predict the test data

In [ ]:
# Instantiate classifiers

lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, **cat_best_params)
logreg_clf = LogisticRegression(random_state=42, **logreg_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
svm_clf = SVC(probability=True, random_state=42, **svm_best_params)

# Create pipelines with PCA and classifiers
pipelines = {
    
    'LightGBM': Pipeline([('pca', PCA()), ('clf', lgbm_clf)]),
    'XGBoost': Pipeline([('pca', PCA()), ('clf', xgb_clf)]),
    'CatBoost': Pipeline([('pca', PCA()), ('clf', cat_clf)]),
    'Logistic Regression': Pipeline([('pca', PCA()), ('clf', logreg_clf)]),
    'Random Forest': Pipeline([('pca', PCA()), ('clf', rf_clf)]),
    'SVM': Pipeline([('pca', PCA()), ('clf', svm_clf)]),
}



In [ ]:
cv_results = []

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
    cv_results.append((name, scores.mean(), scores.std()))

cv_results.sort(key=lambda x: x[1], reverse=True)

print("Classifier performance using cross-validation:")
for name, mean_score, std_dev in cv_results:
    print(f"{name}: {mean_score:.4f} (+/- {std_dev:.4f})")



Stacking classifier of PCA analysis

In [ ]:
base_classifiers = [
    
    ('LightGBM', lgbm_clf),
    ('XGBoost', xgb_clf),
    ('CatBoost', cat_clf),
    ('Logistic Regression', logreg_clf),
    ('Random Forest', rf_clf),
    ('SVM', svm_clf),
    
]

meta_classifier = XGBClassifier(random_state=42)
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)


In [ ]:
stacking_scores = cross_val_score(stacking_clf, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
print("Stacking Classifier performance:")
print(f"Mean recall: {stacking_scores.mean():.4f}, Standard deviation: {stacking_scores.std():.4f}")


In [ ]:
stacking_clf.fit(X_train, y_train)
y_pred = stacking_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)
generate_cm(y_val, stacking_clf.predict(X_val), "Stacking Classifier")
generate_cm(y_test, stacking_clf.predict(X_test), "Stacking Classifier")


In [ ]:
#count the values of y array
from collections import Counter
print(Counter(y_test))
